In [ ]:
from matplotlib import pyplot
import pandas as pd
import warnings
import numpy as np

from summer.utils import ref_times_to_dti

from autumn.core.plots.utils import REF_DATE
from autumn.core.project import get_project
from autumn.core.utils.display import pretty_print

from autumn.core.project import (
    Project,
    ParameterSet,
    build_rel_path,
    get_all_available_scenario_paths,
)

In [ ]:
pyplot.style.use("ggplot")
warnings.filterwarnings("ignore")
region = "wpro_new-zealand"
project = get_project("sm_sir", region)

In [ ]:
baseline_params = project.param_set.baseline
custom_params = project.param_set.baseline
model = project.run_baseline_model(custom_params)

In [ ]:
def smooth_data(target_data):
    """
    removing the zeroes in the data to remove day of the week reporting effect
    """
    # convert series to NumPy array
    data_values = target_data.loc['2020-09-08':'2023-01-08'].to_numpy()
    
    #calculate number of consecutive zeroes
    zero_indices = np.argwhere(data_values == 0)
    for count, value in enumerate(zero_indices):
        i = value
        num_zeros= 0 
        if data_values[i-1] > 0:
            while data_values[i] == 0:            
                num_zeros = num_zeros + 1
                i = i+1
            number_of_zeros = num_zeros
            if number_of_zeros == 1:
                no_of_zeros = number_of_zeros+1
                smoothed_value = data_values[i]/no_of_zeros
                data_values[i-1] = smoothed_value
                data_values[i] = smoothed_value

            if number_of_zeros > 1 :
                no_of_zeros = number_of_zeros+1
                smoothed_value = data_values[i]/no_of_zeros
                data_values[value[0]:value[0]+(number_of_zeros+1)] = np.repeat(smoothed_value,(number_of_zeros+1))
    smoothed_series = pd.Series(data_values)              
    return smoothed_series
             

In [ ]:
targets_dict = {
    t.data.name: pd.Series(t.data.values, index=ref_times_to_dti(model.ref_date, t.data.index)) for 
    t in project.calibration.targets
}

notifications_ts = targets_dict["notifications"]
smoothed_notifications = smooth_data(notifications_ts)

print(smoothed_notifications)

In [ ]:
fig = pyplot.figure(figsize=(15, 12))

axis = fig.add_subplot(1, 1, 1)

#notifications_ts.plot(ax=axis, style='.')
smoothed_notifications.plot(ax=axis, style='.')